In [1]:
# breakhistf2v026.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
  
%matplotlib inline

def read_csv(csvnamepath, filename):
     

    # read from csv file
    images, labels = [], []
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

 
def load_breakhis(dirc, mode='train'):
    classdir2label={}
    filename="tf2breakhisCSV"
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename )  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
@tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y




 

batchsz = 32
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')
db_train = tf.data.Dataset.from_tensor_slices((images, labels))  # 变成个Dataset对象。
db_train = db_train.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)  # map函数图片路径变为内容。
 

# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'val')
db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
db_val = db_val.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)
# create test db
images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'test')
db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
db_test = db_test.map(preprocess1).batch(batchsz)

 
resnetdense = keras.Sequential([
    layers.Conv2D(16,5,3),
    layers.MaxPool2D(3,3),
    layers.ReLU(),
    layers.Conv2D(64,5,3),
    layers.MaxPool2D(2,2),
    layers.ReLU(),
    
    layers.Flatten(),
    #
    layers.Dense(128), 
    layers.Dropout(rate=0.5),
    
    
    
    layers.Dense(64),  
    layers.ReLU(),
    layers.Dense(8)
   ])


 

# 首先创建Resnet18
# resnet = ResNet(8)

resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

# monitor监听器, 连续5个验证准确率不增加，这个事情触发。
 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=100

)

  

 
# 网络的装配。
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

# 完成标准的train，val, test;
# 标准的逻辑必须通过db_val挑选模型的参数，就需要提供一个earlystopping技术，

LOGDIR='log/breakhistf2v026' 
resnetdense.fit(db_train, validation_data=db_val, 
                        validation_freq=10, epochs=3000, 
                        callbacks=[TensorBoard(log_dir=LOGDIR)])   # 1个epoch验证1次。触发了这个事情，提前停止了。
resnetdense.evaluate(db_test)
 
 

    
    
 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  25664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0

297/297 [==============================] - 247s 832ms/step - loss: 0.5316 - accuracy: 0.7963 - val_loss: 0.7044 - val_accuracy: 0.7336
Epoch 131/3000
297/297 [==============================] - 189s 635ms/step - loss: 0.5327 - accuracy: 0.7941
Epoch 132/3000
297/297 [==============================] - 188s 633ms/step - loss: 0.5265 - accuracy: 0.7983
Epoch 133/3000
297/297 [==============================] - 189s 635ms/step - loss: 0.5158 - accuracy: 0.7994
Epoch 134/3000
297/297 [==============================] - 188s 634ms/step - loss: 0.5240 - accuracy: 0.8008
Epoch 135/3000
297/297 [==============================] - 188s 634ms/step - loss: 0.5296 - accuracy: 0.7982
Epoch 136/3000
297/297 [==============================] - 190s 640ms/step - loss: 0.5164 - accuracy: 0.8008
Epoch 137/3000
297/297 [==============================] - 188s 633ms/step - loss: 0.5173 - accuracy: 0.8000
Epoch 138/3000
297/297 [==============================] - 189s 636ms/step - loss: 0.5102 - accuracy: 0.8013
E

297/297 [==============================] - 205s 689ms/step - loss: 0.3479 - accuracy: 0.8636
Epoch 276/3000
297/297 [==============================] - 205s 689ms/step - loss: 0.3348 - accuracy: 0.8674
Epoch 277/3000
297/297 [==============================] - 206s 694ms/step - loss: 0.3428 - accuracy: 0.8641
Epoch 278/3000
297/297 [==============================] - 204s 686ms/step - loss: 0.3304 - accuracy: 0.8663
Epoch 279/3000
297/297 [==============================] - 202s 679ms/step - loss: 0.3517 - accuracy: 0.8609
Epoch 280/3000
297/297 [==============================] - 266s 896ms/step - loss: 0.3324 - accuracy: 0.8690 - val_loss: 0.6410 - val_accuracy: 0.7835
Epoch 281/3000
297/297 [==============================] - 207s 695ms/step - loss: 0.3404 - accuracy: 0.8659
Epoch 282/3000
297/297 [==============================] - 206s 694ms/step - loss: 0.3341 - accuracy: 0.8655
Epoch 283/3000
297/297 [==============================] - 203s 683ms/step - loss: 0.3293 - accuracy: 0.8724
E

Epoch 421/3000
297/297 [==============================] - 190s 639ms/step - loss: 0.2694 - accuracy: 0.8972
Epoch 422/3000
297/297 [==============================] - 192s 647ms/step - loss: 0.2466 - accuracy: 0.9016
Epoch 423/3000
297/297 [==============================] - 192s 647ms/step - loss: 0.2434 - accuracy: 0.9063
Epoch 424/3000
297/297 [==============================] - 190s 639ms/step - loss: 0.2570 - accuracy: 0.8950
Epoch 425/3000
297/297 [==============================] - 190s 640ms/step - loss: 0.2668 - accuracy: 0.8955
Epoch 426/3000
297/297 [==============================] - 190s 639ms/step - loss: 0.2557 - accuracy: 0.8967
Epoch 427/3000
297/297 [==============================] - 189s 636ms/step - loss: 0.2411 - accuracy: 0.9014
Epoch 428/3000
297/297 [==============================] - 189s 638ms/step - loss: 0.2599 - accuracy: 0.8964
Epoch 429/3000
297/297 [==============================] - 190s 640ms/step - loss: 0.2428 - accuracy: 0.9017
Epoch 430/3000
297/297 [====

297/297 [==============================] - 204s 688ms/step - loss: 0.2212 - accuracy: 0.9131
Epoch 495/3000
297/297 [==============================] - 202s 681ms/step - loss: 0.2164 - accuracy: 0.9134
Epoch 496/3000
297/297 [==============================] - 206s 695ms/step - loss: 0.2338 - accuracy: 0.9072
Epoch 497/3000
297/297 [==============================] - 204s 686ms/step - loss: 0.2235 - accuracy: 0.9120
Epoch 498/3000
297/297 [==============================] - 203s 683ms/step - loss: 0.2191 - accuracy: 0.9103
Epoch 499/3000
297/297 [==============================] - 204s 687ms/step - loss: 0.2143 - accuracy: 0.9171
Epoch 500/3000
297/297 [==============================] - 265s 893ms/step - loss: 0.2279 - accuracy: 0.9115 - val_loss: 0.6929 - val_accuracy: 0.7924
Epoch 501/3000
297/297 [==============================] - 206s 694ms/step - loss: 0.2125 - accuracy: 0.9138
Epoch 502/3000
297/297 [==============================] - 206s 695ms/step - loss: 0.2151 - accuracy: 0.9159
E

...

297/297 [==============================] - 205s 692ms/step - loss: 0.0750 - accuracy: 0.9713
Epoch 2445/3000
297/297 [==============================] - 204s 687ms/step - loss: 0.0777 - accuracy: 0.9675
Epoch 2446/3000
297/297 [==============================] - 206s 695ms/step - loss: 0.0817 - accuracy: 0.9671
Epoch 2447/3000
297/297 [==============================] - 204s 686ms/step - loss: 0.0773 - accuracy: 0.9695
Epoch 2448/3000
297/297 [==============================] - 205s 691ms/step - loss: 0.0799 - accuracy: 0.9668
Epoch 2449/3000
297/297 [==============================] - 204s 686ms/step - loss: 0.0760 - accuracy: 0.9708
Epoch 2450/3000
297/297 [==============================] - 268s 901ms/step - loss: 0.0769 - accuracy: 0.9705 - val_loss: 0.9277 - val_accuracy: 0.8161
Epoch 2451/3000
297/297 [==============================] - 205s 690ms/step - loss: 0.0817 - accuracy: 0.9664
Epoch 2452/3000
297/297 [==============================] - 208s 699ms/step - loss: 0.0797 - accuracy: 

297/297 [==============================] - 203s 683ms/step - loss: 0.0693 - accuracy: 0.9733
Epoch 2589/3000
297/297 [==============================] - 205s 690ms/step - loss: 0.0818 - accuracy: 0.9682
Epoch 2590/3000
297/297 [==============================] - 268s 901ms/step - loss: 0.0740 - accuracy: 0.9690 - val_loss: 1.0222 - val_accuracy: 0.8078
Epoch 2591/3000
297/297 [==============================] - 204s 687ms/step - loss: 0.0750 - accuracy: 0.9683
Epoch 2592/3000
297/297 [==============================] - 203s 682ms/step - loss: 0.0741 - accuracy: 0.9706
Epoch 2593/3000
297/297 [==============================] - 203s 684ms/step - loss: 0.0762 - accuracy: 0.9687
Epoch 2594/3000
297/297 [==============================] - 206s 693ms/step - loss: 0.0742 - accuracy: 0.9692
Epoch 2595/3000
297/297 [==============================] - 203s 684ms/step - loss: 0.0735 - accuracy: 0.9712
Epoch 2596/3000
297/297 [==============================] - 203s 685ms/step - loss: 0.0715 - accuracy: 

297/297 [==============================] - 208s 699ms/step - loss: 0.0787 - accuracy: 0.9679
Epoch 2733/3000
297/297 [==============================] - 205s 689ms/step - loss: 0.0683 - accuracy: 0.9717
Epoch 2734/3000
297/297 [==============================] - 206s 693ms/step - loss: 0.0808 - accuracy: 0.9662
Epoch 2735/3000
297/297 [==============================] - 208s 700ms/step - loss: 0.0760 - accuracy: 0.9701
Epoch 2736/3000
297/297 [==============================] - 207s 697ms/step - loss: 0.0712 - accuracy: 0.9726
Epoch 2737/3000
297/297 [==============================] - 205s 690ms/step - loss: 0.0656 - accuracy: 0.9725
Epoch 2738/3000
297/297 [==============================] - 205s 689ms/step - loss: 0.0666 - accuracy: 0.9718
Epoch 2739/3000
297/297 [==============================] - 206s 693ms/step - loss: 0.0775 - accuracy: 0.9698
Epoch 2740/3000
297/297 [==============================] - 267s 900ms/step - loss: 0.0756 - accuracy: 0.9690 - val_loss: 0.9021 - val_accuracy: 

297/297 [==============================] - 207s 698ms/step - loss: 0.0694 - accuracy: 0.9728
Epoch 2877/3000
297/297 [==============================] - 208s 699ms/step - loss: 0.0643 - accuracy: 0.9737
Epoch 2878/3000
297/297 [==============================] - 204s 687ms/step - loss: 0.0871 - accuracy: 0.9651
Epoch 2879/3000
297/297 [==============================] - 205s 689ms/step - loss: 0.0778 - accuracy: 0.9686
Epoch 2880/3000
297/297 [==============================] - 266s 895ms/step - loss: 0.0695 - accuracy: 0.9720 - val_loss: 1.0960 - val_accuracy: 0.8151
Epoch 2881/3000
297/297 [==============================] - 205s 692ms/step - loss: 0.0740 - accuracy: 0.9702
Epoch 2882/3000
297/297 [==============================] - 205s 689ms/step - loss: 0.0778 - accuracy: 0.9703
Epoch 2883/3000
297/297 [==============================] - 205s 690ms/step - loss: 0.0740 - accuracy: 0.9690
Epoch 2884/3000
297/297 [==============================] - 206s 694ms/step - loss: 0.0680 - accuracy: 

[1.0225724744796754, 0.8318584]